# Summary

    az_dream
    + drug_info_release (raw data)
    + drug_info_release_2 (+ PubChem data)
    + drug_info_release_3 (+ CID and SID XRefs)


The following drugs need more **ENSP IDS**:
- 'PIK3CB_PIK3CD': 'PIK3CB,PIK3CD'
- 'PIK3C_2': 'PIK3C*'
- 'PIK3CA_4': 'PIK3CA'
    


# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from common_imports import *

/home/kimlab1/strokach/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
%matplotlib inline

# Functions

In [81]:
def print_stats(df):
    dat.print2('Number of drug -> target tupels:', len(df))
    dat.print2('Number of original drug -> target tuples:', len(df[['drug','target_original']].drop_duplicates()))
    dat.print2('Number of corrected drug -> target tuples:', len(df[['drug','target']].drop_duplicates()))
    dat.print2('Number of HGNC drug -> target tuples:', len(df[['drug','hgnc_name']].drop_duplicates()))
    dat.print2('Number of ENSG drug -> target tuples:', len(df[['drug','ensg']].drop_duplicates()))
    dat.print2('Number of ENST drug -> target tuples:', len(df[['drug','enst']].drop_duplicates()))
    dat.print2('Number of ENSP drug -> target tuples:', len(df[['drug','ensp']].drop_duplicates()))

In [82]:
# Save results to the database
def save_to_db_1(df, table_name):
    engine = sa.create_engine('{}/az_dream_2015'.format(os.environ['BIODB_CONNECTION_STR']))

    df.to_sql(table_name, engine, if_exists='replace', index=False)
    
    engine.execute('create index a on {} (drug(255), hgnc_name(255))'.format(table_name))
    engine.execute('create index b on {} (drug(255), ensg(255))'.format(table_name))
    engine.execute('create index c on {} (drug(255), enst(255))'.format(table_name))
    engine.execute('create unique index d on {} (drug(255), ensp(255))'.format(table_name))

    engine.execute('create index e on {} (hgnc_name(255), drug(255))'.format(table_name))
    engine.execute('create index f on {} (ensg(255), drug(255))'.format(table_name))
    engine.execute('create index g on {} (enst(255), drug(255))'.format(table_name))
    engine.execute('create unique index h on {} (ensp(255), drug(255))'.format(table_name))

    engine.execute('create index i on {} (drug(255), ensp_idx)'.format(table_name))
    engine.execute('create index j on {} (ensp_idx, drug(255))'.format(table_name))
    
def save_to_db_2(df, table_name):
    engine = sa.create_engine('{}/az_dream_2015'.format(os.environ['BIODB_CONNECTION_STR']))

    df.to_sql(table_name, engine, if_exists='replace', index=False)
    
    engine.execute('create unique index a on {} (drug(255), hgnc_name(255))'.format(table_name))
    engine.execute('create unique index b on {} (hgnc_name(255), drug(255))'.format(table_name))

# Load data

In [83]:
# Read text file
drug_info_release = pd.read_csv(
    '../downloads/challenge_data/drug_synergy_data/drug_info_release.csv/'
    'Drug_info_release.csv', sep=','
)
drug_info_release_columns = list(drug_info_release.columns)

In [84]:
display(drug_info_release.head())
print(drug_info_release.shape)

,ChallengeName,Target(Official Symbol),HBA,cLogP,HBD,Lipinski,SMILES or PubChem ID,MW
0,ADAM17,ADAM17,NaN,NaN,NaN,NaN,NaN,NaN
1,AKT,"AKT1, AKT2, AKT3",8,1.18,5,0,c1cc(ccc1[C@H](CCO)NC(=O)C2(CCN(CC2)c3c4cc[nH]...,428.9
2,AKT_1,AKT*,6,3.24,3,0,c1ccc(cc1)c2cc3c(ccn4c3n[nH]c4=O)nc2c5ccc(cc5)...,407.5
3,AKT_PIK3C,"AKT*,PIK3C*",NaN,NaN,NaN,NaN,NaN,NaN
4,AKT_SGK,"AKT*,SGK*",NaN,NaN,NaN,NaN,NaN,NaN


(119, 8)


In [85]:
engine = sa.create_engine('{}/az_dream'.format(os.environ['BIODB_CONNECTION_STR']))
drug_info_release_3 = pd.read_sql_table('drug_info_release_3', engine)

In [86]:
display(drug_info_release_3.head())
print(drug_info_release_3.shape)

,ChallengeName,Target(Official Symbol),HBA,cLogP,HBD,Lipinski,SMILES or PubChem ID,MW,AtomStereoCount,BondStereoCount,CID,CanonicalSMILES,Charge,Complexity,ConformerCount3D,ConformerModelRMSD3D,CovalentUnitCount,DefinedAtomStereoCount,DefinedBondStereoCount,EffectiveRotorCount3D,ExactMass,FeatureAcceptorCount3D,FeatureAnionCount3D,FeatureCationCount3D,FeatureCount3D,FeatureDonorCount3D,FeatureHydrophobeCount3D,FeatureRingCount3D,Fingerprint2D,HBondAcceptorCount,HBondDonorCount,HeavyAtomCount,IUPACName,InChI,InChIKey,IsomericSMILES,IsotopeAtomCount,MolecularFormula,MolecularWeight,MonoisotopicMass,RotatableBondCount,TPSA,UndefinedAtomStereoCount,UndefinedBondStereoCount,Volume3D,XLogP,XStericQuadrupole3D,YStericQuadrupole3D,ZStericQuadrupole3D,cids,targets,targets_bioentities,enzymes,enzymes_bioentities,transporters,transporters_bioentities,atcs
0,ADAM17,ADAM17,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None
1,AKT,"AKT1, AKT2, AKT3",8,1.18,5,0,c1cc(ccc1[C@H](CCO)NC(=O)C2(CCN(CC2)c3c4cc[nH]...,428.9,1,0,25227436,C1CN(CCC1(C(=O)NC(CCO)C2=CC=C(C=C2)Cl)N)C3=NC=...,0,580,256,1.0,1,1,0,8.2,428.172752,2,0,4,14,4,0,4,AAADceB7sAAEAAAAAAAAAAAAAAAAAWAAAAA8WIAAAAAAAF...,6,4,30,4-amino-N-[(1S)-1-(4-chlorophenyl)-3-hydroxypr...,InChI=1S/C21H25ClN6O2/c22-15-3-1-14(2-4-15)17(...,JDUBGYFRJFOXQC-KRWDZBQOSA-N,C1CN(CCC1(C(=O)N[C@@H](CCO)C2=CC=C(C=C2)Cl)N)C...,0,C21H25ClN6O2,428.91520,428.172752,6,120.0,0,0,319.3,1.7,14.27,5.17,1.08,"25227436,42602260,57750340",None,None,None,None,None,None,None
2,AKT_1,AKT*,6,3.24,3,0,c1ccc(cc1)c2cc3c(ccn4c3n[nH]c4=O)nc2c5ccc(cc5)...,407.5,0,0,24964624,C1CC(C1)(C2=CC=C(C=C2)C3=C(C=C4C(=N3)C=CN5C4=N...,0,760,4,0.8,1,0,0,3.8,407.174610,2,0,1,11,2,0,6,AAADceB7oAAAAAAAAAAAAAAAAABgAQAAAAA8YIEAAAAAAE...,4,2,31,8-[4-(1-aminocyclobutyl)phenyl]-9-phenyl-2H-[1...,InChI=1S/C25H21N5O/c26-25(12-4-13-25)18-9-7-17...,ULDXWLCXEDXJGE-UHFFFAOYSA-N,C1CC(C1)(C2=CC=C(C=C2)C3=C(C=C4C(=N3)C=CN5C4=N...,0,C25H21N5O,407.46714,407.174610,3,83.6,0,0,317.2,3.0,16.28,4.11,1.07,24964624,None,None,None,None,None,None,None
3,AKT_PIK3C,"AKT*,PIK3C*",NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None
4,AKT_SGK,"AKT*,SGK*",NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None


(119, 57)


# Drug to target (HGNC)

In [87]:
multitarget_corrections = {
    # Drugs without proper gene names
    'Azacytidine': 'DNMT1',
    'Chloroquine': 'GSTA2,TNF,TLR9,GST', 
    'Cisplatin':   'SLC31A1,ABCC2,ATP7A,ATP7B,HMGB1,MSH6,MLH1,XRCC1,ERCC1,ERCC2,XPA,MPO,SOD1,GSTM1,NQO1,GSTP1,MT', 
    'Carboplatin': 'SLC31A1,ABCC2,ATP7A,ATP7B,HMGB1,MSH6,MLH1,XRCC1,ERCC1,ERCC2,XPA,MPO,SOD1,GSTM1,NQO1,GSTP1,MT', 
    'Oxaliplatin': 'SLC31A1,ABCC2,ATP7A,ATP7B,HMGB1,MSH6,MLH1,XRCC1,ERCC1,ERCC2,XPA,MPO,SOD1,GSTM1,NQO1,GSTP1,MT', 
    'CarboTaxol': ('SLC31A1,ABCC2,ATP7A,ATP7B,HMGB1,MSH6,MLH1,XRCC1,ERCC1,ERCC2,XPA,MPO,SOD1,GSTM1,NQO1,GSTP1,MT'
                   'BCL2,TUBB1,NR1I2,MAP4,MAP2,MAPT'),
    'FOLFIRI': 'TYMS,TOP1,TOP1MT', 
    'FOLFOX': 'TYMS,SLC31A1,ABCC2,ATP7A,ATP7B,HMGB1,MSH6,MLH1,XRCC1,ERCC1,ERCC2,XPA,MPO,SOD1,GSTM1,NQO1,GSTP1,MT',
}

In [88]:
drugs_to_targets_list = [
    (drug, target.strip())
    for (drug, targets) 
    in drug_info_release[['ChallengeName', 'Target(Official Symbol)']].values
    for target in multitarget_corrections.get(drug, targets).split(',')    
]
drugs_to_targets = pd.DataFrame(drugs_to_targets_list, columns=['drug', 'target_original'])

In [89]:
# Corrections to the provided targets
target_corrections = {
    # Corrent typos in gene names
    'BRAF_mut': 'BRAF',
    'BRAF_V600E': 'BRAF',
    'CD19 antibody': 'CD19',
    'VEGFR2': 'KDR',
    'NIAP': 'NAIP',
    'TNFA': 'TNF',
    'NAE2': 'UBA3',
    'TIE2': 'TEK',
    'cMET': 'MET',
    'Gamma secretase': 'APH*',
    'Proteasome': 'PSM*',
}
drugs_to_targets['target'] = drugs_to_targets['target_original'].apply(lambda x: target_corrections.get(x,x))

In [90]:
duplicates = drugs_to_targets[
    (drugs_to_targets.duplicated(subset=['drug', 'target'], keep=False))
]
duplicates

,drug,target_original,target
35,BRAF_VEGFR2,VEGFR2,KDR
36,BRAF_VEGFR2,KDR,KDR
193,NAE,NAE2,UBA3
194,NAE,UBA3,UBA3
245,TKI_2,VEGFR2,KDR
246,TKI_2,TIE2,TEK
247,TKI_2,KDR,KDR
248,TKI_2,TEK,TEK


In [91]:
# Remove duplicate drug -> target
drugs_to_targets = drugs_to_targets.drop_duplicates(subset=['drug', 'target'])

In [92]:
# Get Ensembl IDs for the provided targets
engine = sa.create_engine('{}/az_dream_2015'.format(os.environ['BIODB_CONNECTION_STR']))
sql_query_template = """
select *
from ensembl_sequence_38.hgnc2ensembl
where hgnc_name like '{}'
"""
results = []
for target in drugs_to_targets['target'].drop_duplicates():
    target_like = target.replace('*', '%%').strip()
    sql_query = sql_query_template.format(target_like)
    mapped_targets = pd.read_sql_query(sql_query, engine)
    mapped_targets['target'] = target
    results.append(mapped_targets)
targets_mapped = pd.concat(results, ignore_index=True)

In [93]:
targets_mapped.head()

,hgnc_name,hgnc_acc,hgnc_description,ensg,enst,ensp,target
0,ADAM17,HGNC:195,ADAM metallopeptidase domain 17,ENSG00000151694,ENST00000310823,ENSP00000309968,ADAM17
1,ADAM17,HGNC:195,ADAM metallopeptidase domain 17,ENSG00000151694,ENST00000497134,ENSP00000418728,ADAM17
2,ADAM17,HGNC:195,ADAM metallopeptidase domain 17,ENSG00000151694,ENST00000618923,ENSP00000480552,ADAM17
3,AKT1,HGNC:391,v-akt murine thymoma viral oncogene homolog 1,ENSG00000142208,ENST00000554581,ENSP00000451828,AKT1
4,AKT1,HGNC:391,v-akt murine thymoma viral oncogene homolog 1,ENSG00000142208,ENST00000407796,ENSP00000384293,AKT1


In [94]:
drugs_to_targets_mapped = (
    drugs_to_targets
    .merge(targets_mapped, on=['target'])
)

In [95]:
drugs_to_targets_mapped['ensp_idx'] = (
    drugs_to_targets_mapped['ensp']
    .apply(lambda x: int(x[4:]))
)

In [96]:
display(drugs_to_targets_mapped.head())
dat.print2('Number of drug-target pairs:', len(drugs_to_targets_mapped))
dat.print2('Total number of drugs:', len(drugs_to_targets['drug'].drop_duplicates()))
dat.print2('Number of drugs with mapped targets:', len(drugs_to_targets_mapped['drug'].drop_duplicates()))

,drug,target_original,target,hgnc_name,hgnc_acc,hgnc_description,ensg,enst,ensp,ensp_idx
0,ADAM17,ADAM17,ADAM17,ADAM17,HGNC:195,ADAM metallopeptidase domain 17,ENSG00000151694,ENST00000310823,ENSP00000309968,309968
1,ADAM17,ADAM17,ADAM17,ADAM17,HGNC:195,ADAM metallopeptidase domain 17,ENSG00000151694,ENST00000497134,ENSP00000418728,418728
2,ADAM17,ADAM17,ADAM17,ADAM17,HGNC:195,ADAM metallopeptidase domain 17,ENSG00000151694,ENST00000618923,ENSP00000480552,480552
3,AKT,AKT1,AKT1,AKT1,HGNC:391,v-akt murine thymoma viral oncogene homolog 1,ENSG00000142208,ENST00000554581,ENSP00000451828,451828
4,AKT,AKT1,AKT1,AKT1,HGNC:391,v-akt murine thymoma viral oncogene homolog 1,ENSG00000142208,ENST00000407796,ENSP00000384293,384293


Number of drug-target pairs:                                3231
Total number of drugs:                                      119
Number of drugs with mapped targets:                        119


In [97]:
print_stats(drugs_to_targets_mapped)

Number of drug -> target tupels:                            3231
Number of original drug -> target tuples:                   255
Number of corrected drug -> target tuples:                  255
Number of HGNC drug -> target tuples:                       476
Number of ENSG drug -> target tuples:                       543
Number of ENST drug -> target tuples:                       3231
Number of ENSP drug -> target tuples:                       3231


## Analyse missing

In [98]:
missing_drugs = set(drugs_to_targets['drug']) - set(drugs_to_targets_mapped['drug'])
missing_drugs

set()

In [99]:
missing_targets = set(drugs_to_targets['target']) - set(drugs_to_targets_mapped['target'])
missing_targets

{'CHK1', 'DNA', 'GST', 'MT', 'MTBCL2', 'microtubule'}

## Save

In [100]:
save_to_db_1(drugs_to_targets_mapped, 'drug_to_target')
save_to_db_2(drugs_to_targets_mapped[['drug', 'hgnc_name']].drop_duplicates(), 'drug_to_hgnc_target')

# Drug to target (STITCH)

In [101]:
# Get full drug data for drugs which are missing targets
df = drug_info_release_3[drug_info_release_3['CID'].notnull()].copy()
df['CID'] = df['CID'].apply(lambda x: str(int(float(x))))
display(df.head())
print(df.shape)

,ChallengeName,Target(Official Symbol),HBA,cLogP,HBD,Lipinski,SMILES or PubChem ID,MW,AtomStereoCount,BondStereoCount,CID,CanonicalSMILES,Charge,Complexity,ConformerCount3D,ConformerModelRMSD3D,CovalentUnitCount,DefinedAtomStereoCount,DefinedBondStereoCount,EffectiveRotorCount3D,ExactMass,FeatureAcceptorCount3D,FeatureAnionCount3D,FeatureCationCount3D,FeatureCount3D,FeatureDonorCount3D,FeatureHydrophobeCount3D,FeatureRingCount3D,Fingerprint2D,HBondAcceptorCount,HBondDonorCount,HeavyAtomCount,IUPACName,InChI,InChIKey,IsomericSMILES,IsotopeAtomCount,MolecularFormula,MolecularWeight,MonoisotopicMass,RotatableBondCount,TPSA,UndefinedAtomStereoCount,UndefinedBondStereoCount,Volume3D,XLogP,XStericQuadrupole3D,YStericQuadrupole3D,ZStericQuadrupole3D,cids,targets,targets_bioentities,enzymes,enzymes_bioentities,transporters,transporters_bioentities,atcs
1,AKT,"AKT1, AKT2, AKT3",8,1.18,5,0,c1cc(ccc1[C@H](CCO)NC(=O)C2(CCN(CC2)c3c4cc[nH]...,428.9,1,0,25227436,C1CN(CCC1(C(=O)NC(CCO)C2=CC=C(C=C2)Cl)N)C3=NC=...,0,580,256,1.0,1,1,0,8.2,428.172752,2,0,4,14,4,0,4,AAADceB7sAAEAAAAAAAAAAAAAAAAAWAAAAA8WIAAAAAAAF...,6,4,30,4-amino-N-[(1S)-1-(4-chlorophenyl)-3-hydroxypr...,InChI=1S/C21H25ClN6O2/c22-15-3-1-14(2-4-15)17(...,JDUBGYFRJFOXQC-KRWDZBQOSA-N,C1CN(CCC1(C(=O)N[C@@H](CCO)C2=CC=C(C=C2)Cl)N)C...,0,C21H25ClN6O2,428.915200,428.172752,6,120.0,0,0,319.3,1.7,14.27,5.17,1.08,"25227436,42602260,57750340",None,None,None,None,None,None,None
2,AKT_1,AKT*,6,3.24,3,0,c1ccc(cc1)c2cc3c(ccn4c3n[nH]c4=O)nc2c5ccc(cc5)...,407.5,0,0,24964624,C1CC(C1)(C2=CC=C(C=C2)C3=C(C=C4C(=N3)C=CN5C4=N...,0,760,4,0.8,1,0,0,3.8,407.174610,2,0,1,11,2,0,6,AAADceB7oAAAAAAAAAAAAAAAAABgAQAAAAA8YIEAAAAAAE...,4,2,31,8-[4-(1-aminocyclobutyl)phenyl]-9-phenyl-2H-[1...,InChI=1S/C25H21N5O/c26-25(12-4-13-25)18-9-7-17...,ULDXWLCXEDXJGE-UHFFFAOYSA-N,C1CC(C1)(C2=CC=C(C=C2)C3=C(C=C4C(=N3)C=CN5C4=N...,0,C25H21N5O,407.467140,407.174610,3,83.6,0,0,317.2,3.0,16.28,4.11,1.07,24964624,None,None,None,None,None,None,None
5,ALK,ALK,8,3.89,2,1,CN1CCN(CC1)C2CCN(CC2)c3ccc(c(c3)OC)Nc4ncc(c(n4...,532.1,0,0,56655374,CN1CCN(CC1)C2CCN(CC2)C3=CC(=C(C=C3)NC4=NC=C(C(...,0,746,413,1.0,1,0,0,8.4,531.251336,2,0,5,15,2,0,6,AAADcfB7oAAEAAAAAAAAAAAAAAAAAWAAAAA8eLECAAAAAF...,7,2,38,5-chloro-4-(1H-indol-3-yl)-N-[2-methoxy-4-[4-(...,InChI=1S/C29H34ClN7O/c1-35-13-15-37(16-14-35)2...,HRYNCLKDKCPYBF-UHFFFAOYSA-N,CN1CCN(CC1)C2CCN(CC2)C3=CC(=C(C=C3)NC4=NC=C(C(...,0,C29H34ClN7O,532.079560,531.251336,6,72.6,0,0,408.2,5.0,28.97,5.80,1.37,56655374,None,None,None,None,None,None,None
7,ALK_IGFR,"ALK, IGF*R",7,4.08,4,0,COc1cc(ccc1Nc2ncc(c(n2)c3c[nH]c4c3cccc4)Cl)N5C...,449.0,0,0,56599293,COC1=C(C=CC(=C1)N2CCC(CC2)N)NC3=NC=C(C(=N3)C4=...,0,605,147,1.0,1,0,0,6.2,448.177837,2,0,4,14,3,0,5,AAADceB7oAAEAAAAAAAAAAAAAAAAAWAAAAA8eIEAAAAAAF...,6,3,32,N-[4-(4-aminopiperidin-1-yl)-2-methoxyphenyl]-...,InChI=1S/C24H25ClN6O/c1-32-22-12-16(31-10-8-15...,GCYIGMXOIWJGBU-UHFFFAOYSA-N,COC1=C(C=CC(=C1)N2CCC(CC2)N)NC3=NC=C(C(=N3)C4=...,0,C24H25ClN6O,448.947900,448.177837,5,92.1,0,0,340.1,4.2,17.12,5.81,1.31,56599293,None,None,None,None,None,None,None
8,AR,AR,6,3.35,1,0,CNC(=O)c1ccc(cc1F)N2C(=S)N(C(=O)C2(C)C)c3ccc(C...,464.4,0,0,15951529,CC1(C(=O)N(C(=S)N1C2=CC(=C(C=C2)C(=O)NC)F)C3=C...,0,839,50,0.8,1,0,0,5.6,464.093010,4,0,0,9,1,1,3,AAADceB7scBAAAAAAAAAAAAAAAAAAWAAAAAwYAAAAAAAAA...,8,1,32,"4-[3-[4-cyano-3-(trifluoromethyl)phenyl]-5,5-d...",InChI=1S/C21H16F4N4O2S/c1-20(2)18(31)28(12-5-4...,WXCXUHSOUPDCQV-UHFFFAOYSA-N,CC1(C(=O)N(C(=S)N1C2=CC(=C(C=C2)C(=O)NC)F)C3=C...,0,C21H16F4N4O2S,464.435953,464.093010,3,109.0,0,0,336.1,3.6,19.13,3.82,0.90,"15951529,71580408,71603303,71603305,71603307,7...",Androgen receptor,BE0000132,"Cytochrome P450 2B6,Cytochrome P450 2C19,Cytoc...","BE0003549,BE0003536,BE0002887,BE0002793,BE0002...",Multidrug resistance protein 1,BE0001032,L02BB04


(78, 57)


In [102]:
# Get STITCH target data for drugs which are missing targets
engine = sa.create_engine('{}/stitch_4_0'.format(os.environ['BIODB_CONNECTION_STR']))
sql_query = """
select *
from chemical_sources cs
join protein_chemical_links pcl on (pcl.chemical = cs.flat_chemical)
where protein like '9606.%%'
and source_name = 'PC'
and source_id in ('{}')
""".format("', '".join(str(int(f)) for f in df['CID']))
df2 = pd.read_sql_query(sql_query, engine)

In [103]:
df2['taxid'], df2['protein_id'] = (
    list(zip(*df2['protein'].apply(lambda x: x.split('.'))))
)

In [104]:
display(df2.head())
display(df2.dtypes)

,flat_chemical,stereo_chemical,source_name,source_id,chemical,protein,experimental,databaseID,textmining,combined_score,taxid,protein_id
0,CID110127622,CID010127622,PC,10127622,CID110127622,9606.ENSP00000072869,0,0,0,152,9606,ENSP00000072869
1,CID110127622,CID010127622,PC,10127622,CID110127622,9606.ENSP00000078429,0,0,0,181,9606,ENSP00000078429
2,CID110127622,CID010127622,PC,10127622,CID110127622,9606.ENSP00000085219,0,0,0,162,9606,ENSP00000085219
3,CID110127622,CID010127622,PC,10127622,CID110127622,9606.ENSP00000215832,0,0,700,0,9606,ENSP00000215832
4,CID110127622,CID010127622,PC,10127622,CID110127622,9606.ENSP00000227507,0,0,0,275,9606,ENSP00000227507


flat_chemical      object
stereo_chemical    object
source_name        object
source_id          object
chemical           object
protein            object
experimental        int64
databaseID          int64
textmining          int64
combined_score      int64
taxid              object
protein_id         object
dtype: object

In [105]:
# Get HGNC IDs for targets that were mapped using STITCH
engine = sa.create_engine('{}/ensembl_sequence_38'.format(os.environ['BIODB_CONNECTION_STR']))
sql_query = """
select *
from hgnc2ensembl
where ensp in ('{}')
""".format("', '".join(df2['protein_id'].drop_duplicates()))
targets_mapped_2 = pd.read_sql_query(sql_query, engine)

In [106]:
targets_mapped_2.head()

,hgnc_name,hgnc_acc,hgnc_description,ensg,enst,ensp
0,M6PR,HGNC:6752,mannose-6-phosphate receptor (cation dependent),ENSG00000003056,ENST00000000412,ENSP00000000412
1,ESRRA,HGNC:3471,estrogen-related receptor alpha,ENSG00000173153,ENST00000000442,ENSP00000000442
2,CFTR,HGNC:1884,cystic fibrosis transmembrane conductance regu...,ENSG00000001626,ENST00000003084,ENSP00000003084
3,RALA,HGNC:9839,v-ral simian leukemia viral oncogene homolog A...,ENSG00000006451,ENST00000005257,ENSP00000005257
4,RPAP3,HGNC:26151,RNA polymerase II associated protein 3,ENSG00000005175,ENST00000005386,ENSP00000005386


In [107]:
# Create a drug -> target dataframe
columns = [
    'drug', 'target_original', 'target', 
    'hgnc_name', 'hgnc_acc', 'hgnc_description',
    'ensg', 'enst', 'ensp',
]
column_renames = {
    'ChallengeName': 'drug', 
    'Target(Official Symbol)': 'target_original', 
}
drugs_to_targets_mapped_2 = (
    df.rename(columns=column_renames)
    .merge(df2, left_on=['CID'], right_on=['source_id'])
    .merge(targets_mapped_2, left_on=['protein_id'], right_on=['ensp'])
)
drugs_to_targets_mapped_2['target'] = drugs_to_targets_mapped_2['target_original']
drugs_to_targets_mapped_2 = drugs_to_targets_mapped_2[columns]

In [108]:
drugs_to_targets_mapped_2['ensp_idx'] = (
    drugs_to_targets_mapped_2['ensp']
    .apply(lambda x: int(x[4:]))
)

In [109]:
drugs_to_targets_mapped_2.head()

,drug,target_original,target,hgnc_name,hgnc_acc,hgnc_description,ensg,enst,ensp,ensp_idx
0,AKT,"AKT1, AKT2, AKT3","AKT1, AKT2, AKT3",RTN1,HGNC:10467,reticulon 1,ENSG00000139970,ENST00000267484,ENSP00000267484,267484
1,AKT,"AKT1, AKT2, AKT3","AKT1, AKT2, AKT3",RTN4,HGNC:14085,reticulon 4,ENSG00000115310,ENST00000337526,ENSP00000337838,337838
2,AKT,"AKT1, AKT2, AKT3","AKT1, AKT2, AKT3",BEX1,HGNC:1036,"brain expressed, X-linked 1",ENSG00000133169,ENST00000372728,ENSP00000361813,361813
3,Azacytidine,Methylation,Methylation,BEX1,HGNC:1036,"brain expressed, X-linked 1",ENSG00000133169,ENST00000372728,ENSP00000361813,361813
4,ESR1_3,ESR1,ESR1,BEX1,HGNC:1036,"brain expressed, X-linked 1",ENSG00000133169,ENST00000372728,ENSP00000361813,361813


In [110]:
print_stats(drugs_to_targets_mapped_2)

Number of drug -> target tupels:                            8862
Number of original drug -> target tuples:                   58
Number of corrected drug -> target tuples:                  58
Number of HGNC drug -> target tuples:                       8721
Number of ENSG drug -> target tuples:                       8862
Number of ENST drug -> target tuples:                       8862
Number of ENSP drug -> target tuples:                       8862


## Save

In [111]:
save_to_db_1(drugs_to_targets_mapped_2, 'drug_to_target_stitch')
save_to_db_2(drugs_to_targets_mapped_2[['drug', 'hgnc_name']].drop_duplicates(), 'drug_to_hgnc_target_stitch')

# Drug pair to target pair (HGNC)

In [ ]:
engine = sa.create_engine('{}/ensembl_sequence_38'.format(os.environ['BIODB_CONNECTION_STR']))

# Finalize

In [112]:
os.makedirs(theNotebook, exist_ok=True)

In [113]:
with open(op.join(theNotebook, 'done.lock'), 'w'):
    pass